In [2]:
import pandas as pd

In [1]:
ruta_rar = '/Users/Leon/Downloads/aclImdb/train/pos'


In [3]:
import gensim.downloader as api
from nltk.tokenize import word_tokenize
import numpy as np
import os

/Users/Leon/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:

# Cargar embeddings preentrenados
word_vectors = api.load("glove-wiki-gigaword-100")

def average_word_vector(text, model, dim=100):
    words = word_tokenize(text.lower())
    vecs = [model[w] for w in words if w in model]
    return np.mean(vecs, axis=0) if vecs else np.zeros(dim)



In [5]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import os

def load_imdb_data(path):
    texts = []
    labels = []
    for sentiment in ['pos', 'neg']:
        dir_path = os.path.join(path, sentiment)
        for filename in os.listdir(dir_path):
            with open(os.path.join(dir_path, filename), encoding="utf-8") as f:
                texts.append(f.read())
                labels.append(1 if sentiment == 'pos' else 0)
    return texts, labels

# Cargar datos de entrenamiento
train_path = "/Users/Leon/Library/CloudStorage/GoogleDrive-leoactuario@ciencias.unam.mx/Mi unidad/maestria/ia/segundo semestre/mineria de textos/análisis de sentimientos con vectores de documentos/aclImdb/train"
all_reviews, labels = load_imdb_data(train_path)




# Preparar datos
documents = [TaggedDocument(words=word_tokenize(text.lower()), tags=[str(i)])
             for i, text in enumerate(all_reviews)]

# Entrenar modelo
model_d2v = Doc2Vec(documents, vector_size=100, window=5, min_count=2, workers=4, epochs=20)

# Obtener vectores
vectors_d2v = [model_d2v.infer_vector(doc.words) for doc in documents]


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(all_reviews)

svd = TruncatedSVD(n_components=100)
X_lsa = svd.fit_transform(X_tfidf)


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def train_and_evaluate(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return accuracy_score(y_test, y_pred)

acc_avg = train_and_evaluate(vectors_avg, labels)
acc_d2v = train_and_evaluate(vectors_d2v, labels)
acc_lsa = train_and_evaluate(X_lsa, labels)

print("Accuracy - Word Vector Average:", acc_avg)
print("Accuracy - Doc2Vec:", acc_d2v)
print("Accuracy - TFIDF + SVD (LSA):", acc_lsa)


NameError: name 'vectors_avg' is not defined

In [7]:
pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 10.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os, numpy as np, gensim.downloader as api
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Descargar embeddings GloVe preentrenados
word_vectors = api.load("glove-wiki-gigaword-100")

# Cargar dataset IMDB desde carpeta local
def load_imdb_data(path):
    texts, labels = [], []
    for sentiment in ['pos', 'neg']:
        dir_path = os.path.join(path, sentiment)
        for fname in os.listdir(dir_path):
            with open(os.path.join(dir_path, fname), encoding="utf-8") as f:
                texts.append(f.read())
                labels.append(1 if sentiment == 'pos' else 0)
    return texts, labels

# Cambia esta ruta según la ubicación del dataset
all_reviews, labels = load_imdb_data("/Users/Leon/Library/CloudStorage/GoogleDrive-leoactuario@ciencias.unam.mx/Mi unidad/maestria/ia/segundo semestre/mineria de textos/análisis de sentimientos con vectores de documentos/aclImdb/train")

# Crear vectores promedio usando GloVe
vectors_avg = [np.mean([word_vectors[w] for w in word_tokenize(text.lower()) if w in word_vectors], axis=0) if any(w in word_vectors for w in word_tokenize(text.lower())) else np.zeros(100) for text in all_reviews]

# Entrenar y evaluar un clasificador
X_train, X_test, y_train, y_test = train_test_split(vectors_avg, labels, test_size=0.2, random_state=42)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
accuracy = accuracy_score(y_test, clf.predict(X_test))
print("Accuracy con vectores promedio de GloVe:", accuracy)


/Users/Leon/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Accuracy con vectores promedio de GloVe: 0.7998
